In [2]:
import numpy as np
from numpy import kron, eye as I, diag
import matplotlib.pyplot as plt

from ndgsp import Graph

from scipy.spatial.distance import pdist, squareform

np.set_printoptions(precision=3, linewidth=500, threshold=500, suppress=True, edgeitems=5)

import sys
sys.path.append('..')

from utils.plotting import to_colors


In [8]:
def get_exogenous_tex():
    """
    Create the TikZ code for plotting the exogenous variables plot
    """

    np.random.seed(1)

    x = 2 * np.random.uniform(0, 10, size=10)
    y = 2 * np.random.uniform(0, 10, size=10)

    xx = x - x.mean()
    yy = y - y.mean()

    edges  = np.array([(0, 9),(0, 8),(0, 7),(0, 6),(0, 1),(1, 9),(1, 7),(1, 3),(3, 7),(3, 5), 
                       (6, 7),(5, 6),(6, 8),(2, 5),(2, 6),(2, 4),(4, 6),(4, 8),(8, 9), (7, 5)])


    A = np.zeros((10, 10))

    A[edges[:, 0], edges[:, 1]] = 1
    A = A + A.T

    L = np.diag(A.sum(0)) - A 

    lam, U = np.linalg.eigh(L)

    H = U @ diag(np.exp(-lam)) @ U.T

    def make_section(i, t, offset):

        np.random.seed(i)

        s = np.random.choice(np.arange(10), size=4)

        cols = to_colors(H @ np.random.normal(size=10), cmap='autumn')

        x = np.random.normal(size=4)

        tex = f"\\draw ({-1.5 + offset},2) node{{$t = {t}$}}; \n"
        tex += f"\\draw ({-3.5 + offset},0) node{{ $ \\mathbf{{x}}_{t} = \\begin{{bmatrix}} {x[0]:.2f} \\\\[0.1cm] {x[1]:.2f} \\\\[0.1cm] {x[2]:.2f} \\\\[0.1cm] {x[3]:.2f} \\end{{bmatrix}}$ }};\n"
        tex +=  f"\\draw ({-1.6 + offset},0) node{{$\\mathbf{{y}}_{t} = $}};\n\n"

        for i in range(10):

            if i in s:
                tex += f'\\node[circle, draw, fill={{rgb,255: red,255; green,255; blue,255}}, minimum size=1mm] ({i + j * 10}) at ({offset + xx[i] / 6:.2f}, {yy[i] / 6:.2f}) {{}};\n'

            else:
                tex += f'\\node[circle, draw, fill={{rgb,255: red,{int(cols[i][1:3], base=16)}; green,{int(cols[i][3:5], base=16)}; blue,{int(cols[i][5:], base=16)}}}, minimum size=1mm] ({i + j * 10}) at ({offset + xx[i] / 6:.2f}, {yy[i] / 6:.2f}) {{}};\n'

        tex += '\n'

        for e1, e2 in edges:
            tex += f'\draw[gray, very thin] ({e1 + j * 10}) to ({e2 + j * 10});\n'
 
        return tex + '\n'

    all_tex = """\\documentclass[crop,tikz]{standalone} \n\\usepackage{tikz, amsmath, amssymb} \n\\usetikzlibrary{positioning, shapes.geometric} \n\\begin{document} \n\\begin{tikzpicture}\n"""

    for j, t in enumerate(['1', '2', 'T']):

        if j == 0:
            o = 0
        elif j == 1:
            o = 7
        else:
            o = 15

        all_tex += make_section(j, t, o)

    all_tex += '\\draw (1.3, 0) node{$,$}; \n \\draw (8.9, 0) node{$, \\quad\\quad \\dots $};\n\n'

    all_tex += "\\end{tikzpicture}\n\\end{document} "

    with open('exogenous/exogenous.tex', 'w') as f:
        f.write(all_tex)

    

In [9]:
get_exogenous_tex()

In [10]:
def get_endogenous_tex():
    """
    Create the TikZ code for plotting the RNC diagram
    """

    np.random.seed(1)

    x = 2 * np.random.uniform(0, 10, size=10)
    y = 2 * np.random.uniform(0, 10, size=10)

    xx = x - x.mean()
    yy = y - y.mean()

    edges  = np.array([(0, 9),(0, 8),(0, 7),(0, 6),(0, 1),(1, 9),(1, 7),(1, 3),(3, 7),(3, 5), 
                       (6, 7),(5, 6),(6, 8),(2, 5),(2, 6),(2, 4),(4, 6),(4, 8),(8, 9), (7, 5)])


    A = np.zeros((10, 10))

    A[edges[:, 0], edges[:, 1]] = 1
    A = A + A.T

    L = np.diag(A.sum(0)) - A 

    lam, U = np.linalg.eigh(L)

    H = U @ diag(np.exp(-lam)) @ U.T

    def make_section(i, t, offset):

        np.random.seed(i)

        s = np.random.choice(np.arange(10), size=4)

        cols = to_colors(H @ np.random.normal(size=10), cmap='autumn')

        x = np.random.normal(size=4)

        tex = f"\\draw ({0 + offset},3) node{{$t = {t}$}}; \n"
        tex +=  f"\\draw ({-2.3 + offset},0) node{{$\\mathbf{{y}}_{t} = $}};\n\n"

        for i in range(10):

            if i in s:
                tex += f'\\node[circle, draw, fill={{rgb,255: red,255; green,255; blue,255}}, minimum size=2mm] ({i + j * 10}) at ({offset + xx[i] / 4:.2f}, {yy[i] / 4:.2f}) {{}};\n'

            else:
                tex += f'\\node[circle, draw, fill={{rgb,255: red,{int(cols[i][1:3], base=16)}; green,{int(cols[i][3:5], base=16)}; blue,{int(cols[i][5:], base=16)}}}, minimum size=2mm] ({i + j * 10}) at ({offset + xx[i] / 4:.2f}, {yy[i] / 4:.2f}) {{}};\n'

        tex += '\n'

        for e1, e2 in edges:
            tex += f'\draw[gray, very thin] ({e1 + j * 10}) to ({e2 + j * 10});\n'

        for xi, yi in [(0.8, 2.3), (-2, 1.2), (0.2, 1.1), (-0.7, -0.4), (1.45, 0), (2, 1.7), (-0.8, -2.3), (-1.6, -1.5), (0.5, -1.8), (2, -1.1)]:

            x_ = np.abs(np.random.normal(size=3))
            tex += f'\\draw ({xi + offset}, {yi}) node{{\\scalebox{{0.7}}{{$[{x_[0]:.1f}, {x_[1]:.1f}, {x_[2]:.1f}]$}}}};\n'
 
        return tex + '\n'

    all_tex = """\\documentclass[crop,tikz]{standalone} \n\\usepackage{tikz, amsmath, amssymb} \n\\usetikzlibrary{positioning, shapes.geometric} \n\\begin{document} \n\\begin{tikzpicture}\n"""

    for j, t in enumerate(['1', '2', 'T']):

        if j == 0:
            o = 0
        elif j == 1:
            o = 7
        else:
            o = 15

        all_tex += make_section(j, t, o)


    all_tex += "\draw (2.3, 0) node{$,$};\n" 
    all_tex += '\draw (10.2, 0) node{$, \quad\quad\quad \dots $};\n'

    all_tex += "\\end{tikzpicture}\n\\end{document} "

    with open('endogenous/endogenous.tex', 'w') as f:
        f.write(all_tex)

In [11]:
get_endogenous_tex()